In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import io
import os
import re
import time
import unicodedata

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
#import wandb
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental import preprocessing

from deepcomedy.models.transformer import *
from deepcomedy.preprocessing import load_verses

## 1. Data loading and preprocessing

In [6]:
input_file = "data/divina_textonly.txt"
target_file = "data/divina_syll_textonly.txt"

The `load_verses` function loads the file, splits it into verses, prepends the start_symbol and appends the end_symbol to each verse, then pads each verse to the lenght of the longest verse so that the tensor can be fed to our model.

In [7]:
raw_input_text, input_text, input_tokenizer = load_verses(
    input_file, char_level=True, pad=True
)
raw_target_text, target_text, target_tokenizer = load_verses(
    target_file, char_level=True, pad=True
)

In [8]:
print("Length of input text: {} characters".format(len(raw_input_text)))
print("Length of target text: {} characters".format(len(raw_target_text)))

Length of input text: 578077 characters
Length of target text: 892871 characters


In [9]:
input_vocab = sorted(set(input_tokenizer.word_index.keys()))
target_vocab = sorted(set(target_tokenizer.word_index.keys()))
input_vocab_size = len(input_vocab)
target_vocab_size = len(target_vocab)

In [10]:
print("Input vocab size: {}".format(input_vocab_size))
print("Target vocab size: {}".format(target_vocab_size))

Input vocab size: 80
Target vocab size: 81


In [11]:
input_train, input_test, target_train, target_test = train_test_split(
    input_text, target_text
)

In [12]:
tokenizer = target_tokenizer

## 2. The Transformer model


The dataset is created by grouping the lines in batches and by shuffling them.

Each input's line is in correspondence with its target.

In [13]:
BUFFER_SIZE = len(input_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_train) // BATCH_SIZE
num_layers = 6
d_model = 256
dff = 1024
num_heads = 8
dropout_rate = 0.1

vocab_size = (
    len(tokenizer.word_index) + 1
)  # the +1 is added to take into account the padding token

max_length_targ, max_length_inp = target_text.shape[1], input_text.shape[1]

dataset = tf.data.Dataset.from_tensor_slices((input_train, target_train)).shuffle(
    BUFFER_SIZE
)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [14]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=vocab_size,
    target_vocab_size=vocab_size,
    pe_input=1000,
    pe_target=1000,
    rate=dropout_rate,
)

## 3. Training

In [15]:
transformer_trainer = TransformerTrainer(
    transformer, checkpoint_save_path="./checkpoints/char-level-syll"
)

In [16]:
transformer_trainer.train(dataset, 100)

Epoch 1 Batch 0 Loss 5.1282 Accuracy 0.0010
Epoch 1 Batch 50 Loss 3.9281 Accuracy 0.1416
Epoch 1 Batch 100 Loss 3.4953 Accuracy 0.1773
Epoch 1 Batch 150 Loss 3.3091 Accuracy 0.1917
Epoch 1 Loss 3.2570 Accuracy 0.1973
Time taken for 1 epoch: 52.15 secs

Epoch 2 Batch 0 Loss 2.6296 Accuracy 0.2787
Epoch 2 Batch 50 Loss 2.3678 Accuracy 0.3292
Epoch 2 Batch 100 Loss 2.2139 Accuracy 0.3562
Epoch 2 Batch 150 Loss 2.1165 Accuracy 0.3737
Epoch 2 Loss 2.0924 Accuracy 0.3784
Time taken for 1 epoch: 37.12 secs

Epoch 3 Batch 0 Loss 1.8348 Accuracy 0.4276
Epoch 3 Batch 50 Loss 1.7847 Accuracy 0.4383
Epoch 3 Batch 100 Loss 1.7414 Accuracy 0.4489
Epoch 3 Batch 150 Loss 1.7035 Accuracy 0.4577
Epoch 3 Loss 1.6929 Accuracy 0.4604
Time taken for 1 epoch: 37.11 secs

Epoch 4 Batch 0 Loss 1.6039 Accuracy 0.4755
Epoch 4 Batch 50 Loss 1.5442 Accuracy 0.4986
Epoch 4 Batch 100 Loss 1.5169 Accuracy 0.5063
Epoch 4 Batch 150 Loss 1.4951 Accuracy 0.5117
Epoch 4 Loss 1.4882 Accuracy 0.5136
Time taken for 1 epoch: 

To train the decoder we use teacher forcing, calculating the loss between the predicted logits and the real id of the character.

## 4. Syllabification

We define the *evaluate* function to preprocess the sentence in input to the encoder and to get the predicted ids of the translation.

The ids of the translation are obtained by applying *argmax* to the predicted logits of the decoder.

We begin feeding the decoder with the id of the start symbol and, at each new step, we pass to the decoder the sequence it has just thrown out.

The translation stops when the end symbol is reached.

In [17]:
def evaluate(sentence, max_length=200):

    encoder_input = [tokenizer.word_index[i] for i in list(map(str, sentence))]
    encoder_input = tf.keras.preprocessing.sequence.pad_sequences(
        [encoder_input], maxlen=max_length, padding="post"
    )
    encoder_input = tf.convert_to_tensor(encoder_input)

    output = tf.convert_to_tensor([tokenizer.word_index["^"]])
    output = tf.expand_dims(output, 0)
    result = ""

    for i in range(max_length):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, output
        )

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(
            encoder_input,
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask,
        )

        # select the last character from the seq_len dimension
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.argmax(predictions, axis=-1)

        # concatenate the predicted_id to the output which is given to the decoder as its input.
        output = tf.concat(
            [tf.cast(output, dtype=tf.int32), tf.cast(predicted_id, dtype=tf.int32)],
            axis=-1,
        )
        result += tokenizer.index_word[predicted_id.numpy()[0][0]] + " "

        # return the result if the predicted_id is equal to the end token
        if predicted_id == tokenizer.word_index["$"]:
            break

    # output.shape (1, tokens)

    return result

In [18]:
def print_translation(sentence, result, ground_truth):
    print(f'{"Input:":15s}: {sentence}')
    print(f'{"Prediction":15s}: {result}')
    print(f'{"Ground truth":15s}: {ground_truth}')

In [20]:
sentence = "^E come l’aere, quand’ è ben pïorno,$"
ground_truth = "|E |co|me |l’ ae|re, |quan|d’ è |ben |pï|or|no,"


translated_text = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : ^E come l’aere, quand’ è ben pïorno,$
Prediction     : | C e | d n p a | e t | g i a | l a | v e b | m i r | g e | z a | r e ,   | L o n | v u r u | v u e | z a | r e , $ 
Ground truth   : |E |co|me |l’ ae|re, |quan|d’ è |ben |pï|or|no,


In [24]:
sentence = "^stasera mi butto, mi butto con te$"
ground_truth = ""

translated_text = evaluate(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : ^stasera mi butto, mi butto con te$
Prediction     : | c | s i | c a | l i e | p o m | m s n | v e | p o | e z | s n e | d n e | d r e | s a u | p o | z o $ 
Ground truth   : 
